In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd
import xarray as xr

In [5]:
import h5py
import json

In [6]:
from paleopy import WR, markov

In [7]:
def select_season(data, season='DJF', complete=True, start = 1948, end = 2021, rm_leap=False):
    from calendar import monthrange
    """
    Select a season from data
    data must be a Pandas Series or DataFrame with a datetime index
    """
        
    seasons_params = {}
    seasons_params['DJF'] = (12,2)
    seasons_params['JFM'] = (1,3)
    seasons_params['FMA'] = (2,4)
    seasons_params['MAM'] = (3,5)
    seasons_params['AMJ'] = (4,6)
    seasons_params['MJJ'] = (5,7)
    seasons_params['JJA'] = (6,8)
    seasons_params['JAS'] = (7,9)
    seasons_params['ASO'] = (8,10)
    seasons_params['SON'] = (9,11)
    seasons_params['OND'] = (10,12)
    seasons_params['NDJ'] = (11,1)
    seasons_params['Warm Season (Dec. - May)'] = (12, 5)
    seasons_params['Cold Season (Jun. - Nov.)'] = (6, 11)
    seasons_params['Year (Jan. - Dec.)'] = (1, 12)
    seasons_params['Hydro. year (Jul. - Jun.)'] = (7, 6)    
        
    ### defines the selector 
    selector = ((data.index.month >= seasons_params[season][0]) | (data.index.month <= seasons_params[season][1]))
    
    ### selects
    data = data[selector]
    
    ### if complete == True, we only select COMPLETE seasons 
    data = data.truncate(before='%s-%s-1' % (start, seasons_params[season][0]),\
                   after='%s-%s-%s' % (end, seasons_params[season][1], monthrange(end,seasons_params[season][1])[1] ))
    
    if rm_leap: 
        data[(data.index.month == 2) & (data.index.day == 29)] = np.nan
        data.dropna(inplace=True)
    
    return data

In [8]:
lseason = ['AMJ',
 'ASO',
 'DJF',
 'FMA',
 'JAS',
 'JFM',
 'JJA',
 'MAM',
 'MJJ',
 'NDJ',
 'OND',
 'SON',
 'Cold Season (Jun. - Nov.)',
 'Warm Season (Dec. - May)',
 'Hydro. year (Jul. - Jun.)',
 'Year (Jan. - Dec.)']

In [9]:
with open('../../jsons/WRs.json', 'r') as f: 
    WR_config = json.load(f)

### loads the 9 NZ types 

#### see analyse_clusters_hires_ERA5.ipynb in `/home/nicolasf/research/CPP/WRs/notebooks/Weather_and_Climate_paper`

In [7]:
nz_9_types = pd.read_csv("/media/nicolasf/END19101/data/PICT/datasets/Types/NZ_9_CRs_ERA5.csv", index_col=0, parse_dates=True) 

In [8]:
nz_9_types

,type,class
time,,
1979-01-01,LSW,3
1979-01-02,LSW,3
1979-01-03,LSE,1
1979-01-04,LSE,1
1979-01-05,HS,2
...,...,...
2019-12-27,HW,7
2019-12-28,HW,7
2019-12-29,HW,7


In [70]:
types = ['LSE', 'HS', 'LSW', 'HSE', 'LNE', 'L', 'HW', 'LNW', 'H']

In [77]:
f = h5py.File('../outputs/simulations_9_NZ_CRs.hdf5', mode='a')

for season in lseason: 
    print(f"processing {season} .... \n")
    # calculates the probabilities over the climatological period (1981 - 2010)
    kseason = select_season(nz_9_types, start=1981, end=2010, season=season, rm_leap=False)
    probs = markov.get_probs(kseason['class'].values, np.arange(1, len(types)+1))
    probs = pd.Series(probs, index=types)
    classes, transition_matrix = markov.get_transition_probs(kseason['type'])
    probs = probs.reindex(classes)
    dict_classes, sim2D = markov.simulate_2D(classes, probs.values, transition_matrix, N=len(kseason), P=1000)
    probs = np.empty((len(classes), sim2D.shape[1]))
    for i in range(sim2D.shape[1]): 
        p = markov.get_probs(sim2D[:,i], np.arange(len(classes)))
        probs[:,i] = p
    f["/{}/probs".format(season)] = probs
    f["/{}/probs".format(season)].attrs['shape'] = '(class, simulation)'
    f["/{}/probs".format(season)].attrs['classes'] = ','.join(list(dict_classes.values()))
    del(probs, p)

processing AMJ .... 

processing ASO .... 

processing DJF .... 

processing FMA .... 

processing JAS .... 

processing JFM .... 

processing JJA .... 

processing MAM .... 

processing MJJ .... 

processing NDJ .... 

processing OND .... 

processing SON .... 

processing Cold Season (Jun. - Nov.) .... 

processing Warm Season (Dec. - May) .... 

processing Hydro. year (Jul. - Jun.) .... 

processing Year (Jan. - Dec.) .... 



In [79]:
f.close()

In [80]:
!cp ../outputs/simulations_9_NZ_CRs.hdf5 /media/nicolasf/END19101/data/PICT/datasets/Types/.

In [81]:
f = h5py.File("/media/nicolasf/END19101/data/PICT/datasets/Types/simulations_9_NZ_CRs.hdf5")

In [82]:
f

<HDF5 file "simulations_9_NZ_CRs.hdf5" (mode r)>

In [83]:
f.keys()

<KeysViewHDF5 ['AMJ', 'ASO', 'Cold Season (Jun. - Nov.)', 'DJF', 'FMA', 'Hydro. year (Jul. - Jun.)', 'JAS', 'JFM', 'JJA', 'MAM', 'MJJ', 'NDJ', 'OND', 'SON', 'Warm Season (Dec. - May)', 'Year (Jan. - Dec.)']>

In [84]:
f['DJF']['probs'].attrs['classes']

'H,HS,HSE,HW,L,LNE,LNW,LSE,LSW'

In [85]:
f['AMJ']['probs'].attrs['classes']

'H,HS,HSE,HW,L,LNE,LNW,LSE,LSW'

In [86]:
f['DJF']['probs'][()].shape

(9, 1000)

### Ross Sea Types 

In [11]:
Ross_Sea_types = pd.read_csv("/media/nicolasf/END19101/data/PICT/datasets/Types/Ross_Sea_6_clusters_Cohen.csv", index_col=0, parse_dates=True) 

In [12]:
types = Ross_Sea_types.type.unique()

In [13]:
types

array(['R', 'L', 'LBA', 'LR', 'LA', 'Z'], dtype=object)

In [14]:
Ross_Sea_types

,class,type
time,,
1979-01-01,4,R
1979-01-02,4,R
1979-01-03,5,L
1979-01-04,6,LBA
1979-01-05,4,R
...,...,...
2021-12-27,4,R
2021-12-28,4,R
2021-12-29,4,R


In [15]:
f = h5py.File('/media/nicolasf/END19101/data/PICT/datasets/Types/simulations_Ross_Sea_6_clusters_Cohen.hdf5', mode='a')

for season in lseason: 
    print(f"processing {season} .... \n")
    # calculates the probabilities over the climatological period (1981 - 2010)
    kseason = select_season(Ross_Sea_types, start=1981, end=2010, season=season, rm_leap=False)
    probs = markov.get_probs(kseason['class'].values, np.arange(1, len(types)+1))
    probs = pd.Series(probs, index=types)
    classes, transition_matrix = markov.get_transition_probs(kseason['type'])
    probs = probs.reindex(classes)
    dict_classes, sim2D = markov.simulate_2D(classes, probs.values, transition_matrix, N=len(kseason), P=1000)
    probs = np.empty((len(classes), sim2D.shape[1]))
    for i in range(sim2D.shape[1]): 
        p = markov.get_probs(sim2D[:,i], np.arange(len(classes)))
        probs[:,i] = p
    f["/{}/probs".format(season)] = probs
    f["/{}/probs".format(season)].attrs['shape'] = '(class, simulation)'
    f["/{}/probs".format(season)].attrs['classes'] = ','.join(list(dict_classes.values()))

processing AMJ .... 

processing ASO .... 

processing DJF .... 

processing FMA .... 

processing JAS .... 

processing JFM .... 

processing JJA .... 

processing MAM .... 

processing MJJ .... 

processing NDJ .... 

processing OND .... 

processing SON .... 

processing Cold Season (Jun. - Nov.) .... 

processing Warm Season (Dec. - May) .... 

processing Hydro. year (Jul. - Jun.) .... 

processing Year (Jan. - Dec.) .... 



In [16]:
f.close()